In [1]:
import pandas as pd
from finmath.brazilian_bonds.government_bonds import LTN, NTNF
from finmath.termstructure.curve_models import CurveBootstrap, NelsonSiegelSvensson
from calendars import DayCounts
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
%matplotlib inline


def format_graph(ax):
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0, decimals=2))
    plt.grid(b=True, which='major', color='#666666', linestyle='-')
    plt.minorticks_on()
    plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
    plt.show()


############ Anbima Data ################

ltn_anbima_yields = [2.0580, 2.3885, 2.9904, 3.4463, 4.0148, 4.4847, 4.9137,
                     5.25, 5.7519, 6.115, 6.4247]
ltn_anbima_prices = [997.013474, 990.769777, 981.006668, 969.939968, 955.861261, 940.724187, 923.868863,
                     907.503999, 874.939934, 842.628298, 810.360245]
ltn_maturities = ['2021-04-01', '2021-07-01', '2021-10-01', '2022-01-01', '2022-04-01', '2022-07-01', '2022-10-01',
                  '2023-01-01', '2023-07-01', '2024-01-01', '2024-07-01']

ntnf_anbima_yields = [5.1132, 6.2152, 6.8692, 7.3165, 7.6390]
ntnf_anbima_prices = [1094.209749, 1134.782985, 1154.793279, 1164.388278, 1167.734580]
ntnf_maturities = ['2023-01-01', '2025-01-01', '2027-01-01','2029-01-01', '2031-01-01']

In [2]:
######## LTN ##########
ref_date = pd.to_datetime('2021-02-05').date()
dc = DayCounts('bus/252', calendar='cdr_anbima')

dt_ltn_maturities = [pd.to_datetime(maturity).date() for maturity in ltn_maturities]
ltns = dict()
print('LTN')
for anbima_yield, maturity, anbima_price in zip(ltn_anbima_yields, dt_ltn_maturities, ltn_anbima_prices):
    ltns[maturity] = LTN(expiry=maturity, rate=anbima_yield/100, principal=1000, ref_date=ref_date)
    print(f'{ltns[maturity].expiry} {ltns[maturity].price} {anbima_price} '
          f'{ltns[maturity].rate_from_price(price=anbima_price)*100:.4f} {anbima_yield:.4f}')

LTN
2021-04-01 997.013474 997.013474 2.0580 2.0580
2021-07-01 990.769777 990.769777 2.3885 2.3885
2021-10-01 981.006668 981.006668 2.9904 2.9904
2022-01-01 969.939968 969.939968 3.4463 3.4463
2022-04-01 955.861261 955.861261 4.0148 4.0148
2022-07-01 940.724187 940.724187 4.4847 4.4847
2022-10-01 923.868863 923.868863 4.9137 4.9137
2023-01-01 907.503999 907.503999 5.2500 5.2500
2023-07-01 874.939934 874.939934 5.7519 5.7519
2024-01-01 842.628298 842.628298 6.1150 6.1150
2024-07-01 810.360245 810.360245 6.4247 6.4247


In [3]:
######## NTN-F ##########
dt_ntnf_maturities = [pd.to_datetime(maturity).date() for maturity in ntnf_maturities]
ntnfs = dict()
print('NTN-F')
for anbima_yield, maturity, anbima_price in zip(ntnf_anbima_yields, dt_ntnf_maturities, ntnf_anbima_prices):
    ntnfs[maturity] = NTNF(expiry=maturity, rate=anbima_yield/100, principal=100000, ref_date=ref_date)
    print(f'{ntnfs[maturity].expiry} {ntnfs[maturity].price} {anbima_price*100} '
          f'{ntnfs[maturity].rate_from_price(price=anbima_price*100)*100:.4f} {anbima_yield:.4f}')

NTN-F
2023-01-01 109420.9749 109420.97490000002 5.1132 5.1132
2025-01-01 113478.2985 113478.2985 6.2152 6.2152
2027-01-01 115479.32789999999 115479.3279 6.8692 6.8692
2029-01-01 116438.8278 116438.82779999998 7.3165 7.3165
2031-01-01 116773.458 116773.458 7.6390 7.6390


In [4]:
######## Flat Forward Curve ##########
# print('Flat Forward Curve')
# ltn_prices = list()
# ltn_cash_flows = list()
# for mat, bond in ltns.items():
#     ltn_prices.append(bond.price)
#     ltn_cash_flows.append(pd.Series(index=[mat], data=[bond.principal]))
#
# ntnf_prices = list()
# ntnf_cash_flows = list()
# for mat, bond in ntnfs.items():
#     ntnf_prices.append(bond.price)
#     ntnf_cash_flows.append(bond.cash_flows)
#
# all_prices = ltn_prices+ntnf_prices
# all_cash_flows = ltn_cash_flows+ntnf_cash_flows
#
# bz_curve = CurveBootstrap(prices=all_prices,
#                           cash_flows=all_cash_flows,
#                           ref_date=ref_date)
#
# unique_maturities = set(dt_ltn_maturities + dt_ntnf_maturities)
# unique_maturities = sorted([dc.busdateroll(dt_date, 'modifiedfollowing') for dt_date in unique_maturities])
# flat_forward_curve = pd.Series(data=bz_curve.zero_curve.values, index=unique_maturities)

In [5]:
######## Nelson Siegel Svensson Curve ##########
# print('Nelson Siegel Svensson Curve')
# lambdas = [2.2648, 0.3330]
# nelson_siegel = NelsonSiegelSvensson(prices=all_prices,
#                                      cash_flows=all_cash_flows,
#                                      ref_date=ref_date,
#                                      lambdas=lambdas)
#
# nss_curve = pd.Series(index=unique_maturities, dtype='float64')
# betas = nelson_siegel.betas
# for dt in unique_maturities:
#     year_fraction = dc.tf(ref_date, dt)
#     nss_curve[dt] = nelson_siegel.rate_for_ytm(betas, lambdas, year_fraction)

In [6]:
######## DI1 Curve ##########
# print('DI1 Curve')
# path = 'D:/Pedro/OneDrive/MPE Insper/Renda Fixa/'
# di_df = pd.read_excel(f'{path}DI_table_20210205.xlsx')
# di_df['Maturity'] = [dc.busdateroll(dt_date, 'modifiedfollowing') for dt_date in di_df['Maturity'].values]
#
# di_df.set_index(keys=['Maturity'], inplace=True)
# di_df['Last Price'] = di_df['Last Price'] / 100
# di_series = pd.Series(data=di_df['Last Price'].values, index=di_df.index)

In [7]:
######## All Curves ##########
# all_curves = pd.DataFrame(index=di_series.index, columns=['DI1', 'Flat Forward', 'NSS'])
# all_curves['DI1'] = di_series.values
#
# for dt in all_curves.index:
#     year_fraction = dc.tf(ref_date, dt)
#     all_curves.at[dt, 'Flat Forward'] = bz_curve.rate_for_date(year_fraction)
#     all_curves.at[dt, 'NSS'] = nelson_siegel.rate_for_ytm(betas, lambdas, year_fraction)

In [8]:
######## Flat Forward x NSS ##########
# ax = all_curves[['Flat Forward', 'NSS']].plot(figsize=(15,10), fontsize=16, marker='o')
# format_graph(ax)

In [9]:
######## Flat Forward x DI1 ##########
# ax = all_curves[['Flat Forward', 'DI1']].plot(figsize=(15,10), fontsize=16, marker='o')
# format_graph(ax)

In [10]:
######## NSS x DI1 ##########
# ax = all_curves[['NSS', 'DI1']].plot(figsize=(15,10), fontsize=16, marker='o')
# format_graph(ax)

In [11]:
###### Arbitrage ##########
# bond_maturity = pd.to_datetime('2024-01-01').date()
# tenor_date = dc.busdateroll(pd.to_datetime('2024-01-01').date(), 'modifiedfollowing')
# di1_rate = all_curves[all_curves.index==tenor_date]['DI1']
# principal = 1000000
# bond_rate = all_curves[all_curves.index==tenor_date]['NSS'][0]
# ltn_2024 = LTN(expiry=bond_maturity, rate=bond_rate, ref_date=ref_date,
#                principal=principal)
# loan_at_future = ltn_2024.price*((1+di1_rate[0])**dc.tf(ref_date, bond_maturity))
# yearly_spread = (principal/loan_at_future)**(1/dc.tf(ref_date, bond_maturity))-1


# print(f'Buy Bond with Maturity at {bond_maturity} and principal of {principal:,.2f} @{bond_rate:.4%}')
# print(f'Bond {bond_maturity} price: {ltn_2024.price:,.2f}')
# print(f'Take a loan of {ltn_2024.price:,.2f} @{di1_rate[0]:.4%}')
# print(f'Gain the yearly spread of {yearly_spread:.4%}')